In [ ]:
import pydest

In [ ]:
from pprint import pprint

In [ ]:
destiny = pydest.Pydest("9a46d60a11174668977a76a28b729dd7")

In [ ]:
import time

In [ ]:
start = time.time()
json = await destiny.api.search_destiny_player(-1, 'aadharna')
# await destiny.close()
end = time.time()
print(f"{end - start} seconds to get find my player")

In [ ]:
pprint(json)

In [ ]:
bungoId = await destiny.api.get_membership_data_by_id(4611686018467533572, membership_type=3)

In [ ]:
pprint(bungoId)

In [ ]:
start = time.time()
aaronStats = await destiny.api.get_profile(3, 4611686018467533572, components=[100])
# await destiny.close()
end = time.time()
print(f"{end - start} seconds to get my player profile")

In [ ]:
pprint(aaronStats)

In [ ]:
from collections import defaultdict

In [ ]:
async def getLifetimeStats(playerId, charId, mode):
    stats = await destiny.api.get_historical_stats(-1, 
                                                   playerId, 
                                                   character_id=charId,
                                                   modes=[mode])
    
    gameMode = 'pvpCompetitive' if mode == 69 else 'survival'
    try:
        foo = stats['Response'][gameMode]['allTime']
    except KeyError as e:
        print(stats)
        # import pdb; pdb.set_trace()
        raise ValueError("died to key error")
    life = {}
    for k, v in foo.items():
        life[v['statId']] = v['basic']['value'] 
        
    return life

In [ ]:
import json

In [ ]:
from argparse import Namespace

In [ ]:
import time

In [ ]:
from tqdm import tqdm

In [ ]:
from datetime import datetime
referenceDate = datetime.strptime('2020-11-10T00:00:00Z', "%Y-%m-%dT%H:%M:%SZ")

In [ ]:
names = defaultdict(lambda: {'name':None, 'charId':None})
names[4611686018467284386]['name'] = 'datto'
names[4611686018467284386]['charId'] = 2305843009300009440


player_match_lists = defaultdict(list)

matches = defaultdict(lambda: {'a':{}, 'b':{}})

lifetimeStats = {}
lifetimeStats[4611686018467284386] = await getLifetimeStats(4611686018467284386, 2305843009300009440, 69)

In [ ]:
weapons = {'Auto Rifle':'autoRifle',
 'Combat Bow':"bow",
 'Fusion Rifle':'fusionRifle',
 'Grenade Launcher':'grenadeLauncher',
 'Hand Cannon':'handCannon',
 'Linear Fusion Rifle':'linearFusionRifle',
 'Machine Gun':'machineGun',
 'Pulse Rifle':'pulseRifle',
 'Rocket Launcher':'rocketLauncher',
 'Scout Rifle':'scoutRifle',
 'Shotgun':'shotgun',
 'Sidearm':'sidearm',
 'Sniper Rifle':'sniperRifle',
 'Submachine Gun':'subMachineGun',
 'Sword':'sword',
 'Trace Rifle':'traceRifle'}

In [ ]:
async def getMatches(playerId, charId, count, mode):
    raw = await destiny.api.get_activity_history(3, 
                                                 playerId, 
                                                 character_id=charId,
                                                 count=count,
                                                 mode=mode)
    #from pprint import pprint
    # pprint(raw)
    try:
        foo = raw['Response']['activities']
    except KeyError as e:
        return []
    # pprint(foo)
    matchIds = []
    for v in foo:
        match = int(v['activityDetails']['instanceId'])
        if match in matches:
            continue
        matchIds.append(match)
    
    return matchIds

async def parseMatches(matchIds, mode, beyondLight=False):
    
    for _id in tqdm(matchIds):
        time.sleep(0.05)
        try:
            pgcr = await destiny.api.get_post_game_carnage_report(_id)
        except NameError as e:
            time.sleep(1)
            continue
        
        date = datetime.strptime(pgcr['Response']['period'], "%Y-%m-%dT%H:%M:%SZ")
        if beyondLight and date < referenceDate:
            continue
            
        try:
            for _player in pgcr['Response']['entries']:
                weaponIds = [weaponInfo['referenceId'] for weaponInfo in _player['extended']['weapons']]
        except KeyError as e:
            continue
        
        
        for player in pgcr['Response']['entries']:
            playerId = int(player['player']['destinyUserInfo']['membershipId'])
            playerName = player['player']['destinyUserInfo']['displayName']
            charId = player['characterId']
            
            player_match_lists[playerId].extend([_id])
            if playerId not in names:
                names[playerId]['name'] = playerName
                names[playerId]['charId'] = charId

            baz = {}
            values = {k:v['basic']['value'] for k, v  in player['values'].items()}
            extendedStats = {k:v['basic']['value'] for k, v  in player['extended']['values'].items()}
            weaponIds = [weaponInfo['referenceId'] for weaponInfo in player['extended']['weapons']]
            wKills = [weaponInfo['values']['uniqueWeaponKills']['basic']['value'] 
                     for weaponInfo in player['extended']['weapons']]
            wTypes = []
            for w in weaponIds:
                wInfo = await destiny.decode_hash(w, 'DestinyInventoryItemDefinition')
                wType = wInfo['itemTypeDisplayName']
                wTypes.append(weapons[wType])
            
            flattenedWeaponInfo = {'numWeaponsUsed':len(player['extended']['weapons']),}
            for i, w in enumerate(wTypes):
                flattenedWeaponInfo[w + 'kills'] = wKills[i]
                flattenedWeaponInfo[w + 'Id'] = weaponIds[i]
                
            baz.update(flattenedWeaponInfo)
            baz.update(extendedStats)
            baz.update(values)
            
            a = 17 if mode == 69 else 18
            #  mode 37 has teams of 18 and 19
            #  mode 69 has teams of 17 and 18
            
            if baz['team'] == a:
                teamKey = 'b'
            else:
                teamKey = 'a'
            
            matches[_id][teamKey][playerId] = baz
            matches[_id][teamKey][playerId]['date'] = date
            
            if playerId not in lifetimeStats:
                lifetimeStats[playerId] = await getLifetimeStats(playerId, charId, mode)
            
    return

In [ ]:
# datto: 4611686018467284386, 2305843009300009440
# aaron: 4611686018467533572, 2305843009300337881
matchIds = await getMatches(4611686018467533572, 2305843009300337881, 5, mode=37)
# 84 = ToO
# 69 = PvP Comp playlist

In [ ]:
matchIds

In [ ]:
from copy import deepcopy

In [ ]:
await parseMatches(matchIds, 37, beyondLight=True)

In [ ]:
len(lifetimeStats)

In [ ]:
matches

In [ ]:
len(player_match_lists)

In [ ]:
len(matches)

In [ ]:
len(names)

In [ ]:
sample0 = deepcopy(names)

In [ ]:
newnewNames = {}
for k, v in names.items():
    if k in sample1 or k in sample0:
        continue
    newnewNames[k] = v
sample2 = deepcopy(names)

In [ ]:
print(len(newnewNames), len(newNames), len(names), len(sample0), len(sample1))

In [ ]:
import numpy as np

In [ ]:
initialBatch = deepcopy(newnewNames)

for k, v in initialBatch.items():
    if np.random.rand() > 0.1:
        continue
    if len(player_match_lists[k]) >= 50:
        continue
    matchIds = await getMatches(k, v['charId'], 50, mode=37)
    try:
        await parseMatches(matchIds, 37, beyondLight=True)
    except:
        continue

In [ ]:
len(matches)

In [ ]:
len(player_match_lists)

In [ ]:
import pickle
import os

In [ ]:
def save_obj(obj, folder, name):
    path = os.path.join(folder, name) + '.pkl'
    if os.path.exists(path):
        os.remove(path)
    with open(path, 'wb+') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
        
def load_obj(folder, name):
    path = os.path.join(folder, name)
    try:
        with open(path, 'rb') as f:
            f = pickle.load(f)
    except EOFError:
        f = {"resend":True}
    return f

In [ ]:
save_obj(dict(lifetimeStats), '.', 'lifetimeStats.BL')

In [ ]:
save_obj(dict(matches), '.', 'matches.BL')

In [ ]:
save_obj(dict(names), '.', 'playerIdMap.BL')

In [ ]:
save_obj(dict(player_match_lists), '.', 'playerMatchLists.BL')

In [ ]:
mode = await destiny.decode_hash(1161276682, 'DestinyInventoryItemDefinition')

In [ ]:
pprint(mode)